In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import rasterio

In [5]:
PARK_PATH = 'park.png'
CAM_VIEW_PATH = 'cam_view.jpg'
MIN_MATCH_COUNT = 10

In [6]:
def get_homography_matrix(img1, img2):
    # Initiate SIFT detector
    sift = cv2.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)

    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        return cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    else:
        print("Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT))
        return None

In [7]:
park = cv2.imread(PARK_PATH, 0)
cam_view = cv2.imread(CAM_VIEW_PATH, 0)

h, status = get_homography_matrix(park, cam_view)

print(h)
print(status)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 11841676800 bytes in function 'cv::OutOfMemoryError'
